# MERFISH Data reorganization

In [1]:
import pandas as pd
import numpy as np
import os, sys, time
import re
import shutil
from importlib import reload
# this version doesn't have ChromAn compiled, so directly load from src:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares/ChromAn")
import src

In [9]:
from src.file_io.data_organization import search_fovs_in_folders, Color_Usage
data_home =  r"/lab/weissman_imaging/puzheng"

In [4]:
data_folder = os.path.join(data_home, 'msBrain', '20230830-msBrain_Mop_MF1')

fds, fovs = search_fovs_in_folders(data_folder)


- searching in folder: /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1
-- 20 folders, 398 fovs detected.


In [12]:
# Color usage file marks the organization of imaging files
color_usage_filename = os.path.join(data_folder, 'Analysis', 'color_usage.csv')
color_usage_df = Color_Usage(color_usage_filename)

- load color_usage from file: /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/Analysis/color_usage.csv


['H0M1',
 'H1M2',
 'H2M3',
 'H3M4',
 'H4M5',
 'H5M6',
 'H6M7',
 'H7M8',
 'H8M9',
 'H10M11',
 'H11M12',
 'H12M13']

In [6]:
date = os.path.basename(data_folder).split('-')[0]

sample = 'msTest'
library = 'MO4'

print(date, sample, library)

20230830 msTest MO4


In [7]:
from src.file_io.data_organization import create_folder
# shutter files
ref_image_type = '748_637_477_405_s13_n1000' # ref 58 / 65
polyt_image_type = '748_637_477_s13_n1000'
image_type = '748_637_477_s13_n1000' # ref 36/39

# fiducial beads
#ref_channel = 488
#ref_frame = 13 # start from 0!

# target folder
merfish_data_folder = r'/lab/weissman_imaging/puzheng/MERFISH_data'

# target folder:
target_folder = os.path.join(merfish_data_folder, f'{date}-{sample}_{library}')
create_folder(target_folder)


Creating folder: /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4


In [ ]:
#sel_fovs = []
folder_regexp = r'H([0-9]+)[MRCP]([0-9+])'

ref_round = 0
polyt_round = 8
#fov_ids = np.arange(2)
overwrite = False
remove_source = False

for _fov_id, _fov_name in enumerate(fovs):
    # check if this fov is selected:
    if 'sel_fovs' in locals() and _fov_id not in sel_fovs:
        continue # skip if not selected
    # prceed for the rest:
    for _fd in fds:
        # match round and hyb
        _match = re.search(folder_regexp, os.path.basename(_fd))
        if _match is None:
            continue
        # if not exist in color_usage, skip
        if os.path.basename(_fd) not in list(color_usage_df.index):
            continue
        _round, _region = _match.groups()
        # mutliple files for each image:
        _files = [os.path.join(_fd, _fl) for _fl in os.listdir(_fd) if _fov_name.split(os.extsep)[0] in _fl]
        # reference round specified
        if int(_round) == ref_round:
            _image_type = ref_image_type
        elif int(_round) == polyt_round:
            _image_type = polyt_image_type
        else:
            _image_type = image_type
    
        # target file
        _target_files = [os.path.join(target_folder, f"{_image_type}_{_fov_id}_{_round}{os.extsep}{_fl.split(os.extsep)[-1]}") 
                         for _fl in _files]
        for _fl, _tar_fl in zip(_files, _target_files):
            if remove_source:
                print(f"move {_fl} to {_tar_fl}")
                shutil.move(_fl, _tar_fl)
            else:
                if not os.path.isfile(_tar_fl) or overwrite:
                    print(f"copy {_fl} to {_tar_fl}")
                    shutil.copyfile(_fl, _tar_fl)
                else:
                    print(f"{_tar_fl} already exist, skip")

/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_405_s13_n1000_0_0.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_405_s13_n1000_0_0.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_405_s13_n1000_0_0.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_405_s13_n1000_0_0.off already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_405_s13_n1000_0_0.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_0_1.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_0_1.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_0_1.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477

/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_6.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_6.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_6.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_6.off already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_6.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_7.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_7.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_7.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2_7.off a

/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_11.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_11.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_11.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_11.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_11.off already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_12.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_12.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_5_12.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000

/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_1.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_1.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_1.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_1.off already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_1.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_2.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_2.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_13_2.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000

/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_1.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_2.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_2.off already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_2.power already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_2.xml already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_2.dax already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_3.inf already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_20_3.off already exist, skip
/lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_2

...

copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H3M4/Conv_zscan_339.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_339_3.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H3M4/Conv_zscan_339.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_339_3.power
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H4M5/Conv_zscan_339.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_339_4.off
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H4M5/Conv_zscan_339.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_339_4.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H4M5/Conv_zscan_339.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_339_4.dax
copy /lab/weissman_imaging/puzheng/msBrain/202

copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H0M1/Conv_zscan_340.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_405_s13_n1000_340_0.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H1M2/Conv_zscan_340.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H1M2/Conv_zscan_340.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_1.power
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H1M2/Conv_zscan_340.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_1.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H1M2/Conv_zscan_340.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_1.dax
copy /lab/weissman_imaging/puzheng/msBrain

copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H10M11/Conv_zscan_340.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_10.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H11M12/Conv_zscan_340.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_11.off
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H11M12/Conv_zscan_340.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_11.power
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H11M12/Conv_zscan_340.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_11.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H11M12/Conv_zscan_340.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_340_11.dax
copy /lab/weissman_imaging/puzh

copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H6M7/Conv_zscan_341.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_341_6.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H6M7/Conv_zscan_341.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_341_6.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H7M8/Conv_zscan_341.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_341_7.power
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H7M8/Conv_zscan_341.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_341_7.off
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H7M8/Conv_zscan_341.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_341_7.dax
copy /lab/weissman_imaging/puzheng/msBrain/202

copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H3M4/Conv_zscan_342.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_342_3.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H3M4/Conv_zscan_342.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_342_3.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H3M4/Conv_zscan_342.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_342_3.power
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H3M4/Conv_zscan_342.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_342_3.off
copy /lab/weissman_imaging/puzheng/msBrain/20230830-msBrain_Mop_MF1/H4M5/Conv_zscan_342.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230830-msTest_MO4/748_637_477_s13_n1000_342_4.dax
copy /lab/weissman_imaging/puzheng/msBrain/202

## Check file length

In [17]:
os.listdir(target_folder)

['748_637_545_477_405_s13_n1000_5Hz_24_0.power',
 '748_637_545_477_405_s13_n1000_5Hz_70_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_174_0.xml',
 '748_637_545_477_s13_n1000_5Hz_32_8.dax',
 '748_637_545_477_s13_n1000_5Hz_213_8.off',
 '748_637_545_477_s13_n1000_5Hz_24_8.inf',
 '748_637_545_477_405_s13_n1000_5Hz_215_0.off',
 '748_637_545_477_405_s13_n1000_5Hz_45_0.power',
 '748_637_545_477_405_s13_n1000_5Hz_219_0.off',
 '748_637_545_477_405_s13_n1000_5Hz_104_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_188_0.power',
 '748_637_545_477_405_s13_n1000_5Hz_199_0.inf',
 '748_637_545_477_s13_n1000_5Hz_61_8.power',
 '748_637_545_477_405_s13_n1000_5Hz_251_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_78_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_20_0.xml',
 '748_637_545_477_s13_n1000_5Hz_253_8.dax',
 '748_637_545_477_s13_n1000_5Hz_124_8.xml',
 '748_637_545_477_405_s13_n1000_5Hz_135_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_217_0.xml',
 '748_637_545_477_s13_n1000_5Hz_249_8.xml',
 '748_637_545_477_s1

In [20]:
# double check if filenumber equals to expected:
print(len(os.listdir(target_folder)), len(fds) * len(fovs) * 5)

11475 11475


# Generate data_organization

In [ ]:
## Automatic generate data_organization
from src.file_io.data_organization import Color_Usage

In [ ]:
def generate_dataOrganization():
    

